In [3]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:14 http://security.ubuntu.c

In [4]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-05-23 03:25:25--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2022-05-23 03:25:25 (11.4 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
#aws-fp20-instance.cchbtewatphd.us-west-2.rds.amazonaws.com

# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://myosus32022buk.s3.amazonaws.com/census_estimates.csv"
spark.sparkContext.addFile(url)
CountyPop_df = spark.read.csv(SparkFiles.get("census_estimates.csv"), sep=",", header=True, inferSchema=True)


In [7]:
CountyPop_df.show()

+------------+----------------------------------------+------------------------------------------+---------------------------+------------------------+----+----+----+----+----+----+----+----+----+----+
|         _c0|Revised Population Estimate July 1, 2020|Certified Population Estimate July 1, 2021|Population Change 2020-2021|Percent Change 2020-2021| _c5| _c6| _c7| _c8| _c9|_c10|_c11|_c12|_c13|_c14|
+------------+----------------------------------------+------------------------------------------+---------------------------+------------------------+----+----+----+----+----+----+----+----+----+----+
|     Oregon |                               4,243,791|                                 4,266,560|                     22,769|                   0.54%|null|null|null|null|null|null|null|null|null|null|
|      BAKER |                                  16,721|                                    16,860|                        139|                   0.83%|null|null|null|null|null|null|null|null|n

In [8]:
url ="https://myosus32022buk.s3.amazonaws.com/County%20Product%20Trend_Full%20Data_data.csv"
spark.sparkContext.addFile(url)
CountyProductData_df = spark.read.csv(SparkFiles.get("County Product Trend_Full Data_data.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
CountyProductData_df.show()

+---------+-------------------+---------+----------+----------+------------+-------------------+--------+------------+
|     Date|       Product Type|   County|Month Name|Month Year|Tooltip Date|Market Share County|   Sales|Sales Detail|
+---------+-------------------+---------+----------+----------+------------+-------------------+--------+------------+
|10/1/2016|          Usable MJ|    Baker|   October| 10/1/2016|October 2016|              100.0|58637.51|    58637.51|
|10/1/2016|          Usable MJ|    Baker|   October| 10/1/2016|October 2016|              100.0|   441.0|       441.0|
|10/1/2016|Concentrate/Extract|Clackamas|   October| 10/1/2016|October 2016|              15.14|   803.2|       803.2|
|10/1/2016|    Edible/Tincture|Clackamas|   October| 10/1/2016|October 2016|               9.31|  359.68|      359.68|
|10/1/2016|              Other|Clackamas|   October| 10/1/2016|October 2016|               1.13|    30.0|        30.0|
|10/1/2016|          Usable MJ|Clackamas|   Octo

In [12]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
#aws-fp20-instance.cchbtewatphd.us-west-2.rds.amazonaws.com
jdbc_url="jdbc:postgresql://aws-fp20-instance.cchbtewatphd.us-west-2.rds.amazonaws.com:5432/AWSFP20"
config = {"user":"postgres",
        "password": password,
        "driver":"org.postgresql.Driver"}



Enter database password··········


In [13]:
# Write DataFrame to CountyPop table in RDS
CountyPop_df.write.jdbc(url=jdbc_url, table='CountyPopTest', mode=mode, properties=config)


In [ ]:
# Write DataFrame to CanabisData table in RDS
CountyProductData_df.write.jdbc(url=jdbc_url, table='CanabisData', mode=mode, properties=config)


In [ ]:
# Join the two DataFrame
joined_df = user_data_df.join(user_payment_df, on="username", how="inner")
joined_df.show()


+------------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|    username| id|first_name|last_name|active_user|      street_address|               state|billing_id|        cc_encrypted|
+------------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|   ibearham0|  1|    Cletus|  Lithcow|      false| 78309 Riverside Way|            Virginia|         1|a799fcafe47d7fb19...|
|    wwaller1|  2|       Caz|   Felgat|      false| 83 Hazelcrest Place|             Alabama|         2|a799fcafe47d7fb19...|
|   ichesnut2|  3|     Kerri|  Crowson|      false|      112 Eliot Pass|      North Carolina|         3|a799fcafe47d7fb19...|
|     tsnarr3|  4|   Freddie|    Caghy|      false|     15 Merchant Way|            New York|         4|a799fcafe47d7fb19...|
|   fwherrit4|  5|   Sadella|    Deuss|      false|    079 Acker Avenue|           Tennessee|         5|a799fcafe47d7f

In [ ]:
dropna_df = joined_df.dropna()
dropna_df.show()

+------------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|    username| id|first_name|last_name|active_user|      street_address|               state|billing_id|        cc_encrypted|
+------------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|   ibearham0|  1|    Cletus|  Lithcow|      false| 78309 Riverside Way|            Virginia|         1|a799fcafe47d7fb19...|
|    wwaller1|  2|       Caz|   Felgat|      false| 83 Hazelcrest Place|             Alabama|         2|a799fcafe47d7fb19...|
|   ichesnut2|  3|     Kerri|  Crowson|      false|      112 Eliot Pass|      North Carolina|         3|a799fcafe47d7fb19...|
|     tsnarr3|  4|   Freddie|    Caghy|      false|     15 Merchant Way|            New York|         4|a799fcafe47d7fb19...|
|   fwherrit4|  5|   Sadella|    Deuss|      false|    079 Acker Avenue|           Tennessee|         5|a799fcafe47d7f

In [ ]:
from pyspark.sql.functions import col

cleaned_df = dropna_df.filter(col("active_user") ==True)

In [ ]:
cleaned_df.show()

+-------------+---+----------+-----------+-----------+--------------------+--------------------+----------+--------------------+
|     username| id|first_name|  last_name|active_user|      street_address|               state|billing_id|        cc_encrypted|
+-------------+---+----------+-----------+-----------+--------------------+--------------------+----------+--------------------+
|   fstappard5|  6|    Fraser|    Korneev|       true|  76084 Novick Court|           Minnesota|         6|a799fcafe47d7fb19...|
|   lhambling6|  7|    Demott|     Rapson|       true|    86320 Dahle Park|District of Columbia|         7|a799fcafe47d7fb19...|
|    wheinerte| 15|   Sadella|      Jaram|       true|7528 Waxwing Terrace|         Connecticut|        15|a799fcafe47d7fb19...|
| droughsedgeg| 17|    Hewitt|    Trammel|       true|    2455 Corry Alley|      North Carolina|        17|a799fcafe47d7fb19...|
|    ydudeniei| 19|       Ted|    Knowlys|       true|      31 South Drive|                Ohio| 

In [ ]:
clean_user_df = cleaned_df.select(["id", "first_name", "last_name", "username"])

In [ ]:
clean_billing_df = cleaned_df.select(["billing_id", "street_address", "state", "username"])

In [ ]:
clean_payment_df = cleaned_df.select(["billing_id", "cc_encrypted"])

In [ ]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://osuawsm16db.cchbtewatphd.us-west-2.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"postgres",
        "password": password,
        "driver":"org.postgresql.Driver"}



Enter database password··········


In [ ]:
# Write DataFrame to active_user table in RDS
clean_user_df.write.jdbc(url=jdbc_url, table='active_user', mode=mode, properties=config)

# Write dataframe to billing_info table in RDS
clean_billing_df.write.jdbc(url=jdbc_url, table='billing_info', mode=mode, properties=config)

# Write dataframe to payment_info table in RDS
clean_payment_df.write.jdbc(url=jdbc_url, table='payment_info', mode=mode, properties=config)


